# Embedding model
## bge-m3

In [1]:
from sentence_transformers import SentenceTransformer

In [2]:
model = SentenceTransformer("BAAI/bge-m3")

c:\Users\user\anaconda3\envs\nlp\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [6]:
embedded_vector = model.encode("야 저기 차온다")
embedded_vector

array([ 0.00560306,  0.03283845, -0.04176652, ..., -0.00452853,
       -0.02254617,  0.04927669], dtype=float32)

In [7]:
import pandas as pd

data = [
    "내일 차타고 놀러가자",
    "지금 오는 버스는 어디서 오는 버스야?",
    "차 한잔 하면서 이야기 하시죠",
    "5차 공동구매! 오늘만 세일!",
    "홍차 녹차 중에 어떤 차가 좋으세요?",
]

df = pd.DataFrame(data, columns=['text'])

In [8]:
def get_embedded_vector(text):
    return list(model.encode(text))

df['embedding'] = df.apply(lambda x : get_embedded_vector(x.text), axis = 1)
df

,text,embedding
0,내일 차타고 놀러가자,"[0.0054786922, 0.016985165, -0.02041358, 0.024..."
1,지금 오는 버스는 어디서 오는 버스야?,"[-0.013388522, 0.024189422, -0.025403237, -0.0..."
2,차 한잔 하면서 이야기 하시죠,"[-0.0022157743, 0.010684666, -0.039016657, 0.0..."
3,5차 공동구매! 오늘만 세일!,"[-0.030319413, -0.0011549561, -0.03304448, 0.0..."
4,홍차 녹차 중에 어떤 차가 좋으세요?,"[0.0361152, 0.0046490077, -0.021344075, -0.028..."


In [9]:
import numpy as np

def cos_sim(a, b):
    return a @ b/(np.linalg.norm(a)*np.linalg.norm(b))

In [16]:
def return_answer_candidate(df, query):
    query_embedding = get_embedded_vector(query)
    df["similarity"] = df.embedding.apply(lambda x : cos_sim(np.array(x), np.array(query_embedding)))
    result = df.sort_values("similarity", ascending = False)
    return result

In [17]:
sim_result = return_answer_candidate(df, "야 저기 차 온다")
sim_result

,text,embedding,similarity
0,내일 차타고 놀러가자,"[0.0054786922, 0.016985165, -0.02041358, 0.024...",0.720349
1,지금 오는 버스는 어디서 오는 버스야?,"[-0.013388522, 0.024189422, -0.025403237, -0.0...",0.710821
2,차 한잔 하면서 이야기 하시죠,"[-0.0022157743, 0.010684666, -0.039016657, 0.0...",0.664830
3,5차 공동구매! 오늘만 세일!,"[-0.030319413, -0.0011549561, -0.03304448, 0.0...",0.477791
4,홍차 녹차 중에 어떤 차가 좋으세요?,"[0.0361152, 0.0046490077, -0.021344075, -0.028...",0.441064
